In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.wkt import loads
from shapely import wkb



In [ ]:
target_df = pd.read_parquet(".parquet")
all_df = pd.read_parquet(".parquet")

In [ ]:
if target_df['geometry'].dtype == object:
    target_df['geometry'] = target_df['geometry'].apply(wkb.loads)
if all_df['geometry'].dtype == object:
    all_df['geometry'] = all_df['geometry'].apply(wkb.loads)

In [ ]:
target_gdf = gpd.GeoDataFrame(target_df, geometry='geometry', crs="EPSG:4326")
all_gdf = gpd.GeoDataFrame(all_df, geometry='geometry', crs="EPSG:4326")

In [ ]:
target_gdf = target_gdf.to_crs("EPSG:3857")
all_gdf = all_gdf.to_crs("EPSG:3857")

In [ ]:
for dist in [50, 100, 250, 500]:
    target_gdf[f"building_density_{dist}"] = 0


In [ ]:
batch_size = 1
results = []

for i in range(0, len(target_gdf), batch_size):
    print(f"Processing batch {i}...")
    target_batch = target_gdf.iloc[i:i+batch_size].copy()

    for dist in [50, 100, 250, 500]:
        target_batch[f"building_density_{dist}"] = 0

        buffer_batch = target_batch.copy()
        buffer_batch["geometry"] = buffer_batch.geometry.buffer(dist)

        joined = gpd.sjoin(all_gdf, buffer_batch, how="inner", predicate="within")
        counts = joined.groupby("index_right").size()

        target_batch.loc[counts.index, f"building_density_{dist}"] = counts.values

    results.append(target_batch)

# concatenate all results
target_gdf = pd.concat(results)


In [ ]:
target_gdf = target_gdf.to_crs("EPSG:4326")
print(target_gdf.head())

In [ ]:
target_gdf.to_parquet("try_AP_BD")